##### Navie Dense

In [ ]:
from typing import Any
import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size, out_size, activation) -> None:
        self.activation = activation

        w_shape = (input_size, out_size)
        w_init_value = tf.random.uniform(w_shape, minval=0.0, maxval=1e-1)
        self.W = tf.Variable(w_init_value, "float32")

        b_shape = (out_size,)
        b_init_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_init_value, "float32")

    def __call__(self, inputs) -> Any:
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self):
        return [self.W, self.b]

##### Navie Sequentail

In [ ]:
from typing import Any


class NaiveSequentail:
    def __init__(self, layers) -> None:
        self.layers = layers

    def __call__(self, inputs) -> Any:
        x = inputs

        for layer in self.layers:
            x = layer(x)

        return x
    
    @property
    def weights(self):
        weights = []

        for layer in self.layers:
            weights += layer.weights

        return weights

##### Model

In [ ]:
model = NaiveSequentail([
    NaiveDense(input_size=28*28, out_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, out_size=10, activation=tf.nn.softmax)
])

##### Batch Generator

In [ ]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_siez = 128) -> None:
        assert(len(images) == len(labels))

        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_siez = batch_siez
        self.num_batchs = math.ceil(len(images) / batch_siez)

    def next(self):
        images = self.images[self.index:self.index + self.batch_siez]
        labels = self.labels[self.index:self.index + self.batch_siez]
        self.index += self.batch_siez

        return images, labels

In [ ]:
from tensorflow import keras

optimizer = keras.optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [ ]:
    
def one_traing_steps(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        pre_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_losses = tf.reduce_mean(pre_sample_losses)

        gradients = tape.gradient(average_losses, model.weights)
        update_weights(gradients, model.weights)

        return average_losses

In [ ]:
def fit(model, images, labels, epochs, batch_size = 128):
    for epoch in range(epochs):
        batch_generator = BatchGenerator(images, labels, batch_size)
        for batch_counter in range(batch_generator.num_batchs):
            images_batch, labels_batch = batch_generator.next()
            loss = one_traing_steps(model, images_batch, labels_batch)
            
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

In [ ]:
from tensorflow import keras
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape(60000, 28 * 28)
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape(10000, 28 * 28)
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, 32)

In [ ]:
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()

predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels

print(f"accuracy {matches.mean():.2f}")